# AutoRAG-Allocator - Google Colab Setup

Budget-aware model assignment for compound RAG pipelines.

**Requirements**: Colab Pro with A100 GPU (recommended)


## Step 1: Setup and Installation


In [ ]:
# Option 1: Clone from GitHub (if public repo)
# !git clone https://github.com/yourusername/ece570proj.git

# Option 2: Upload files manually
# If repo is private, use: File > Upload to upload the autorag-allocator folder
# Then uncomment the line below:
# %cd /content/autorag-allocator

# For now, let's set up the directory structure
import os
from pathlib import Path

# Check for project in common locations (including nested structure)
possible_paths = [
    Path('/content/autorag-allocator/ece570proj/autorag-allocator'),  # Nested structure
    Path('/content/autorag-allocator'),  # Direct upload
    Path('/content/ece570proj/autorag-allocator'),  # Git clone
]

project_dir = None
for path in possible_paths:
    if path.exists() and (path / 'requirements.txt').exists():
        project_dir = path
        break

if project_dir:
    os.chdir(project_dir)
    print(f"✅ Found project at: {project_dir}")
    print(f"Working directory: {os.getcwd()}")
else:
    # Create default directory
    project_dir = Path('/content/autorag-allocator')
    project_dir.mkdir(exist_ok=True)
    os.chdir(project_dir)
    print(f"Working directory: {os.getcwd()}")
    print("\n📁 Please upload your autorag-allocator folder to /content/")
    print("   Or clone from GitHub if your repo is public")
    print("\n   To upload: Use Colab's file browser (📁 icon) > Upload")

# Check if we have the files
if project_dir and (project_dir / 'requirements.txt').exists():
    print("\n✅ Found requirements.txt, installing dependencies...")
    %pip install -q -r requirements.txt
else:
    print("\n⚠️  requirements.txt not found. Please upload files first.")
    print("   Expected locations:")
    print("   - /content/autorag-allocator/ece570proj/autorag-allocator/requirements.txt")
    print("   - /content/autorag-allocator/requirements.txt")
    print("   - /content/ece570proj/autorag-allocator/requirements.txt")

# Install FAISS (Colab-compatible version)
# Note: faiss-gpu might not be available in all Colab environments
# We'll try faiss-gpu first, fall back to faiss-cpu if needed
import subprocess
import sys

print("\nInstalling FAISS...")
result = subprocess.run([sys.executable, "-m", "pip", "install", "-q", "faiss-gpu"], 
                       capture_output=True, text=True)
if result.returncode == 0:
    print("✅ Installed faiss-gpu")
else:
    print("⚠️  faiss-gpu not available, installing faiss-cpu (works fine in Colab)")
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "faiss-cpu"], 
                  check=False)
    print("✅ Installed faiss-cpu")

print("\n✅ Installation complete")


## Step 2: Configure API Keys


In [ ]:
import os
from google.colab import userdata

# Method 1: Use Colab secrets (recommended)
try:
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
    print("✅ API keys loaded from Colab secrets")
except Exception as e:
    print(f"⚠️  Could not load from secrets: {e}")
    print("Using direct assignment instead...")
    # Method 2: Set directly (less secure, but works)
    # os.environ['OPENAI_API_KEY'] = 'sk-...'
    # os.environ['GROQ_API_KEY'] = 'gsk_...'

# Verify
has_openai = bool(os.getenv('OPENAI_API_KEY'))
has_groq = bool(os.getenv('GROQ_API_KEY'))
print(f"OPENAI_API_KEY: {'✅ Set' if has_openai else '❌ Missing'}")
print(f"GROQ_API_KEY: {'✅ Set' if has_groq else '❌ Missing'}")

if not (has_openai and has_groq):
    print("\n⚠️  Please set API keys using Colab secrets (🔑 icon in sidebar)")


## Step 3: Verify GPU


In [ ]:
import torch

print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    props = torch.cuda.get_device_properties(0)
    print(f"GPU Memory: {props.total_memory / 1e9:.1f} GB")
    print(f"✅ GPU ready for FAISS acceleration")
else:
    print("⚠️  No GPU detected")
    print("Go to: Runtime > Change runtime type > GPU (A100)")


## Step 4: Test Setup


In [ ]:
import sys
import os
from pathlib import Path

# Add project to path (check nested structure first)
project_paths = [
    '/content/autorag-allocator/ece570proj/autorag-allocator',  # Nested structure
    '/content/autorag-allocator',  # Direct upload
    '/content/ece570proj/autorag-allocator',  # Git clone
    os.getcwd()
]

for path in project_paths:
    if Path(path).exists() and (Path(path) / 'src').exists():
        sys.path.insert(0, path)
        print(f"✅ Added {path} to Python path")
        break
else:
    print("⚠️  Could not find project directory. Please ensure files are uploaded.")

# Test API keys
import os
has_openai = bool(os.getenv('OPENAI_API_KEY'))
has_groq = bool(os.getenv('GROQ_API_KEY'))
print(f"\nAPI Keys Status:")
print(f"  OPENAI_API_KEY: {'✅ Set' if has_openai else '❌ Missing'}")
print(f"  GROQ_API_KEY: {'✅ Set' if has_groq else '❌ Missing'}")

if not (has_openai and has_groq):
    print("  ⚠️  Some API keys missing. Set them in Colab secrets (🔑 icon)")

# Test corpus loading (small sample)
try:
    # Ensure project root is in path (not just the directory itself)
    found_path = None
    for path in project_paths:
        p = Path(path)
        if p.exists() and (p / 'src').exists():
            found_path = str(p)
            if found_path not in sys.path:
                sys.path.insert(0, found_path)
            break
    
    if found_path:
        print(f"\n✅ Project root in path: {found_path}")
        print(f"   Checking for src/data/corpus.py...")
        corpus_file = Path(found_path) / 'src' / 'data' / 'corpus.py'
        if corpus_file.exists():
            print(f"   ✅ Found corpus.py at {corpus_file}")
        else:
            print(f"   ❌ corpus.py not found at {corpus_file}")
    
    # Now try the import
    from src.data.corpus import load_wikipedia_corpus
    
    print("\nTesting corpus loading (100 passages)...")
    corpus = load_wikipedia_corpus(n_passages=100, seed=42)
    print(f"✅ Loaded {len(corpus)} passages")
    print(f"Sample: {corpus[0][:150]}...")
except ImportError as e:
    print(f"\n⚠️  Could not import corpus loader: {e}")
    print("   This might be a path issue. Will work during full experiment.")
    print(f"   Current Python path includes:")
    for p in sys.path:
        if 'autorag' in p or 'ece570' in p:
            print(f"     - {p}")
except Exception as e:
    print(f"\n⚠️  Error loading corpus: {e}")
    print("   This is okay for now - will work during full experiment")
    import traceback
    traceback.print_exc()


## Step 5: Run Full Experiment


In [ ]:
import sys
import os
from pathlib import Path

# Find project directory (check nested structure first)
project_paths = [
    '/content/autorag-allocator/ece570proj/autorag-allocator',  # Nested structure
    '/content/autorag-allocator',  # Direct upload
    '/content/ece570proj/autorag-allocator',  # Git clone
    os.getcwd()
]

project_dir = None
for path in project_paths:
    if Path(path).exists() and (Path(path) / 'src').exists():
        project_dir = path
        sys.path.insert(0, path)
        break

if not project_dir:
    raise RuntimeError("Could not find project directory. Please upload files first.")

os.chdir(project_dir)
print(f"Working directory: {os.getcwd()}")

# Set corpus size (1000 for testing, 10000 for full experiment)
corpus_size = 10000  # Change to 1000 for quick test
os.environ['CORPUS_SIZE'] = str(corpus_size)

print(f"\nRunning experiment with corpus size: {corpus_size}")
print("This may take 30-45 minutes for full corpus...\n")

from experiments.run_experiments import main
main()


## Step 6: View Results


In [ ]:
import json
from pathlib import Path
import os

# Find results file in common locations (including nested structure)
results_paths = [
    Path('/content/autorag-allocator/ece570proj/autorag-allocator/results/full_results.json'),  # Nested
    Path('/content/autorag-allocator/results/full_results.json'),  # Direct
    Path('/content/ece570proj/autorag-allocator/results/full_results.json'),  # Git clone
    Path(os.getcwd()) / 'results' / 'full_results.json'  # Current dir
]

results_file = None
for path in results_paths:
    if path.exists():
        results_file = path
        break

if results_file and results_file.exists():
    with open(results_file, 'r') as f:
        results = json.load(f)
    
    print("=== Experiment Results ===\n")
    
    # NQ-Open Baseline
    nq_base = results.get('nq_baseline', {})
    print(f"NQ-Open Baseline:")
    print(f"  EM: {nq_base.get('em', 0):.1f}%")
    print(f"  F1: {nq_base.get('f1', 0):.1f}%")
    print(f"  Cost: {nq_base.get('cost_cents', 0):.2f}¢/query")
    print(f"  Latency: {nq_base.get('latency_ms', 0):.0f}ms\n")
    
    # NQ-Open Adaptive
    nq_adapt = results.get('nq_adaptive', {})
    best = nq_adapt.get('best', {})
    if best:
        config = best.get('config', {})
        print(f"NQ-Open Adaptive (Best):")
        print(f"  Config: {config.get('R', '?')}/{config.get('G', '?')}/{config.get('V', '?')}")
        print(f"  EM: {best.get('em', 0):.1f}%")
        print(f"  F1: {best.get('f1', 0):.1f}%")
        print(f"  Cost: {best.get('cost_cents', 0):.2f}¢/query")
        print(f"  Latency: {best.get('latency_ms', 0):.0f}ms\n")
        
        # Improvement
        em_improve = best.get('em', 0) - nq_base.get('em', 0)
        cost_improve = nq_base.get('cost_cents', 0) - best.get('cost_cents', 0)
        print(f"Improvement:")
        print(f"  EM: +{em_improve:.1f}%")
        print(f"  Cost: -{cost_improve:.2f}¢/query")
else:
    print("⚠️  Results file not found. Run experiment first.")


## Step 7: Generate Figures


In [ ]:
import sys
import os
from pathlib import Path

# Find project directory (check nested structure first)
project_paths = [
    '/content/autorag-allocator/ece570proj/autorag-allocator',  # Nested structure
    '/content/autorag-allocator',  # Direct upload
    '/content/ece570proj/autorag-allocator',  # Git clone
    os.getcwd()
]

for path in project_paths:
    if Path(path).exists() and (Path(path) / 'src').exists():
        sys.path.insert(0, path)
        break

try:
    from experiments.generate_figures import main as gen_figures
    
    print("Generating figures...")
    gen_figures()
    print("✅ Figures saved to results/ directory")
except ImportError as e:
    print(f"⚠️  Could not import figure generator: {e}")
    print("   Make sure experiments/generate_figures.py exists")
except Exception as e:
    print(f"⚠️  Error generating figures: {e}")


## Step 8: Download Results


In [ ]:
from google.colab import files
from pathlib import Path
import os

# Find results directory (including nested structure)
results_dirs = [
    Path('/content/autorag-allocator/ece570proj/autorag-allocator/results'),  # Nested
    Path('/content/autorag-allocator/results'),  # Direct
    Path('/content/ece570proj/autorag-allocator/results'),  # Git clone
    Path(os.getcwd()) / 'results'  # Current dir
]

results_dir = None
for dir_path in results_dirs:
    if dir_path.exists():
        results_dir = dir_path
        break

if results_dir:
    # Download results JSON
    results_file = results_dir / 'full_results.json'
    if results_file.exists():
        files.download(str(results_file))
        print(f"✅ Downloaded {results_file.name}")
    
    # Download figures
    for fig_file in ['results_comparison.pdf', 'pareto_frontier.pdf', 'profiling_overhead.pdf']:
        fig_path = results_dir / fig_file
        if fig_path.exists():
            files.download(str(fig_path))
            print(f"✅ Downloaded {fig_file}")
    
    print("\n✅ Downloads initiated")
else:
    print("⚠️  Results directory not found")


## Monitor Resources


In [ ]:
# Check disk usage
!df -h /content

# Check GPU memory
!nvidia-smi

# List cached files (check common locations)
import os
from pathlib import Path

cache_dirs = [
    Path('/content/autorag-allocator/ece570proj/autorag-allocator/data/cache'),  # Nested
    Path('/content/autorag-allocator/data/cache'),  # Direct
    Path('/content/ece570proj/autorag-allocator/data/cache'),  # Git clone
    Path(os.getcwd()) / 'data' / 'cache'  # Current dir
]

for cache_dir in cache_dirs:
    if cache_dir.exists():
        print(f"\nCache directory: {cache_dir}")
        !ls -lh {cache_dir}
        break
else:
    print("\nNo cache directory found yet")
